# Rastrigin Function Optimization

## 1. CUDA Parallel Implementation

In [2]:

%%writefile rastrigin_cuda.cu
#include <iostream>
#include <cmath>
#include <curand_kernel.h>
#include <cfloat>
#include <chrono>
#include <fstream>
#include <iomanip>

using namespace std;

// ------------------------------
// PARAMETERS
// ------------------------------
#define POP 256
#define DIM 5
#define MAX_IT 8000
// Rastrigin standard bounds are [-5.12, 5.12]
#define LB -5.12
#define UB 5.12

#define BLOCK_SIZE 256

// ------------------------------
// DEVICE HELPER FUNCTIONS
// ------------------------------
// Generate random double [0, 1]
__device__ double randDouble(curandState* state) {
    return curand_uniform_double(state);
}

// Generate random integer [a, b]
__device__ int randInt(curandState* state, int a, int b) {
    float r = curand_uniform(state);
    return (int)(a + r * (b - a + 0.99999f));
}

// ------------------------------
// FITNESS FUNCTION: RASTRIGIN
// ------------------------------
__device__ double fitness(double* x, int dim) {
    // f(x) = 10d + sum(x_i^2 - 10cos(2*pi*x_i))
    double sum = 10.0 * dim;
    const double PI = 3.14159265358979323846;
    for (int i = 0; i < dim; ++i) {
        sum += x[i] * x[i] - 10.0 * cos(2.0 * PI * x[i]);
    }
    return sum;
}

// ------------------------------
// KERNELS
// ------------------------------

// Initialize Population (Paper Eq. 2)
__global__ void init_kernel(double* pop, double* fitness_vals, curandState* states, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        // Init RNG unique to this thread
        curand_init(seed, idx, 0, &states[idx]);

        // Init Position
        double sol[DIM];
        for (int d = 0; d < DIM; d++) {
            // Eq (2): x = lb + r * (ub - lb)
            sol[d] = LB + randDouble(&states[idx]) * (UB - LB);
            pop[idx * DIM + d] = sol[d];
        }

        // Init Fitness
        fitness_vals[idx] = fitness(sol, DIM);
    }
}

// Main LOA Logic (Section 3.3)
__global__ void loa_kernel(double* pop, double* fitness_vals, double* new_pop, double* new_fitness_vals, curandState* states, int t) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < POP) {
        curandState localState = states[idx];
        double myFit = fitness_vals[idx];

        // Load current position
        double currentPos[DIM];
        for(int d=0; d<DIM; d++) currentPos[d] = pop[idx * DIM + d];

        // ------------------------------------------------
        // Identify Safe Areas (SA_i) - Eq (5)
        // SA_i = {X_k | F_k < F_i}
        // ------------------------------------------------
        int betterCount = 0;
        int betterIdx = -1;

        // Pass 1: Count how many particles are better than me
        for (int j = 0; j < POP; j++) {
            if (fitness_vals[j] < myFit) {
                betterCount++;
            }
        }

        // Pass 2: Randomly select ONE from the safe area set
        if (betterCount > 0) {
            int pick = randInt(&localState, 0, betterCount - 1);
            int current = 0;
            for (int j = 0; j < POP; j++) {
                if (fitness_vals[j] < myFit) {
                    if (current == pick) {
                        betterIdx = j;
                        break;
                    }
                    current++;
                }
            }
        }

        // ------------------------------------------------
        // Strategy Selection - Eq (4)
        // ------------------------------------------------
        double rp = randDouble(&localState);
        double candidate[DIM];

        // Note: If betterIdx is -1 (we are the best), force Phase 2 (Hiding)
        // as there is no "safer" area to escape to.
        bool doEscape = (rp <= 0.5) && (betterIdx != -1);

        if (doEscape) {
            // --- Phase 1: Escaping Strategy (Exploration) ---

            // Get the Selected Safe Area (SSA) position
            double SSA[DIM];
            for(int d=0; d<DIM; d++) SSA[d] = pop[betterIdx * DIM + d];

            for(int d=0; d<DIM; d++) {
                // Eq (6): x_new = x + r * (SSA - I * x)
                double r_ij = randDouble(&localState);
                int I_ij = randInt(&localState, 1, 2);

                candidate[d] = currentPos[d] + r_ij * (SSA[d] - I_ij * currentPos[d]);

                // Bounds Checking
                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        } else {
            // --- Phase 2: Hiding Strategy (Exploitation) ---

            for(int d=0; d<DIM; d++) {
                // Eq (8): x_new = x + (1 - 2r) * (ub - lb) / t
                double r_ij = randDouble(&localState);

                // Note: t is iteration count (1 to MAX_IT)
                candidate[d] = currentPos[d] + (1.0 - 2.0 * r_ij) * (UB - LB) / (double)t;

                // Bounds Checking
                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        }

        // ------------------------------------------------
        // Evaluation & Update - Eq (7) & Eq (9)
        // ------------------------------------------------
        double f_new = fitness(candidate, DIM);

        if (f_new < myFit) {
            // Accept new position
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = candidate[d];
            new_fitness_vals[idx] = f_new;
        } else {
            // Keep old position
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = currentPos[d];
            new_fitness_vals[idx] = myFit;
        }

        // Save RNG state
        states[idx] = localState;
    }
}

// ------------------------------
// MAIN
// ------------------------------
int main() {
    auto t_start = chrono::high_resolution_clock::now();

    // 1. Setup CSV Logging
    ofstream csvFile("rastrigin_data.csv");
    csvFile << "Iteration,Best_Fitness";
    for(int d=0; d<DIM; d++) csvFile << ",x" << d+1;
    csvFile << "\n";
    csvFile << fixed << setprecision(10);

    // 2. Allocate Memory
    double *d_pop, *d_fitness, *d_new_pop, *d_new_fitness;
    curandState *d_states;

    size_t pop_bytes = POP * DIM * sizeof(double);
    size_t fit_bytes = POP * sizeof(double);

    cudaMalloc(&d_pop, pop_bytes);
    cudaMalloc(&d_fitness, fit_bytes);
    cudaMalloc(&d_new_pop, pop_bytes);
    cudaMalloc(&d_new_fitness, fit_bytes);
    cudaMalloc(&d_states, POP * sizeof(curandState));

    // 3. Initialization
    int threads = 128;
    int blocks = (POP + threads - 1) / threads;

    init_kernel<<<blocks, threads>>>(d_pop, d_fitness, d_states, time(NULL));
    cudaDeviceSynchronize();

    // Host buffers for logging
    double h_pop[POP * DIM];
    double h_fitness[POP];
    double globalBestFit = DBL_MAX;

    cout << "Starting LOA Optimization on Rastrigin..." << endl;

    // 4. Main Loop
    for (int it = 1; it <= MAX_IT; it++) {
        // Run Kernel
        loa_kernel<<<blocks, threads>>>(d_pop, d_fitness, d_new_pop, d_new_fitness, d_states, it);
        cudaDeviceSynchronize();

        // Swap Pointers (Double Buffering)
        double *temp_pop = d_pop; d_pop = d_new_pop; d_new_pop = temp_pop;
        double *temp_fit = d_fitness; d_fitness = d_new_fitness; d_new_fitness = temp_fit;

        // --- DATA LOGGING ---
        // Copy back to CPU to find best
        cudaMemcpy(h_fitness, d_fitness, fit_bytes, cudaMemcpyDeviceToHost);
        cudaMemcpy(h_pop, d_pop, pop_bytes, cudaMemcpyDeviceToHost);

        int bestIdx = 0;
        for (int i = 1; i < POP; i++) {
            if (h_fitness[i] < h_fitness[bestIdx]) {
                bestIdx = i;
            }
        }

        if (h_fitness[bestIdx] < globalBestFit) {
            globalBestFit = h_fitness[bestIdx];
        }

        // Write to CSV
        csvFile << it << "," << h_fitness[bestIdx];
        for(int d=0; d<DIM; d++) {
            csvFile << "," << h_pop[bestIdx * DIM + d];
        }
        csvFile << "\n";

        if (it % 1000 == 0) {
            cout << "Iter " << it << " | Best = " << globalBestFit << endl;
        }
    }

    csvFile.close();

    // Final Output
    cout << "\nOptimization Complete.\n";
    cout << "Final Best Solution:\n";

    // Refresh host data one last time to be sure
    cudaMemcpy(h_fitness, d_fitness, fit_bytes, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_pop, d_pop, pop_bytes, cudaMemcpyDeviceToHost);

    int bestIdx = 0;
    for (int i = 1; i < POP; i++) if (h_fitness[i] < h_fitness[bestIdx]) bestIdx = i;

    for (int d = 0; d < DIM; d++) {
        cout << "x" << d+1 << " = " << h_pop[bestIdx * DIM + d] << endl;
    }
    cout << "\nBest Rastrigin Value = " << h_fitness[bestIdx] << endl;

    auto t_end = chrono::high_resolution_clock::now();
    double elapsed = chrono::duration_cast<chrono::duration<double>>(t_end - t_start).count();
    cout << "Execution Time = " << elapsed << " sec" << endl;

    // Cleanup
    cudaFree(d_pop); cudaFree(d_fitness);
    cudaFree(d_new_pop); cudaFree(d_new_fitness);
    cudaFree(d_states);

    return 0;
}

Overwriting rastrigin_cuda.cu


In [3]:
!nvcc -arch=sm_75 rastrigin_cuda.cu -o rastrigin_cuda
!./rastrigin_cuda

Starting LOA Optimization on Rastrigin...
Iter 1000 | Best = 0
Iter 2000 | Best = 0
Iter 3000 | Best = 0
Iter 4000 | Best = 0
Iter 5000 | Best = 0
Iter 6000 | Best = 0
Iter 7000 | Best = 0
Iter 8000 | Best = 0

Optimization Complete.
Final Best Solution:
x1 = -2.29364e-09
x2 = 2.99046e-09
x3 = 4.78108e-10
x4 = -5.4598e-10
x5 = -1.38954e-10

Best Rastrigin Value = 0
Execution Time = 0.963288 sec


## 2. Serial (Sequential) Implementation

In [ ]:
%%writefile rastrigin_serial.cpp
#include <bits/stdc++.h>
using namespace std;

/* ---------------------------------------
      Random Generator
---------------------------------------*/
mt19937 rng(time(NULL));

double randF(double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double rastrigin(const vector<double> &x) {
    int dim = x.size();

    double sum = 10.0 * dim;
    const double PI = 3.14159265358979323846;
    for (int i = 0; i < dim; ++i) {
        sum += x[i] * x[i] - 10.0 * cos(2.0 * PI * x[i]);
    }
    return sum;

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(LB, UB); // Note: Original code used randF(rng,0,1) here but logic was r*(SSA-I*x).
        // Wait, original code: double r = randF(rng,0,1);
        // Let's stick to original logic.
        double r_val = randF(0, 1);
        int I = randInt(1, 2);
        newX[j] = x[j] + r_val * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search)
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(0, 1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population
---------------------------------------*/
vector<vector<double>> init_population(){
    vector<vector<double>> pop(POP, vector<double>(DIM));
    for(int i=0;i<POP;i++)
        for(int d=0;d<DIM;d++)
            pop[i][d] = randF(LB, UB);
    return pop;
}

/* =======================================
             MAIN LOA SERIAL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    vector<vector<double>> pop = init_population();
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=rastrigin(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        for(int i=0;i<POP;i++){

            vector<int> better;
            for(int j=0;j<POP;j++)
                if(fitness[j]<fitness[i]) better.push_back(j);

            int betterIdx=-1;
            if(!better.empty())
                betterIdx = better[randInt(0,(int)better.size()-1)];

            vector<double> candidate;
            if(randF(0,1)<0.5 && betterIdx!=-1)
                candidate = escape(pop[i],pop[betterIdx]);
            else
                candidate = hide(pop[i],it);

            double f = rastrigin(candidate);

            if(f<fitness[i]){
                pop[i]=candidate;
                fitness[i]=f;
            }
            if(f<bestFit){
                bestFit=f;
                bestSol=candidate;
            }
        }

        if(it < 10)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Rastrigin" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Overwriting rastrigin_serial.cpp


In [ ]:
!g++ rastrigin_serial.cpp -o rastrigin_serial
!./rastrigin_serial

Iter 1 | Best = 1117.44
Iter 2 | Best = 219.562
Iter 3 | Best = 219.562
Iter 4 | Best = 128.871
Iter 5 | Best = 61.8016
Iter 6 | Best = 58.0953
Iter 7 | Best = 58.0953
Iter 8 | Best = 36.6651
Iter 9 | Best = 36.6651

Final Best Solution:
x1 = -4.76305e-09
x2 = 3.25239e-09
x3 = -3.07083e-09
x4 = 5.17939e-10
x5 = -1.66585e-09

Best Rastrigin Value = 0

Execution Time = 6.20622 sec


## 3. OpenMP Parallel Implementation

In [ ]:

%%writefile rastrigin_omp.cpp
#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

/* ---------------------------------------
      Thread-safe Random Generator
---------------------------------------*/
double randF(mt19937 &rng, double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(mt19937 &rng, int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double rastrigin(const vector<double> &x) {
    int dim = x.size();

    double sum = 10.0 * dim;
    const double PI = 3.14159265358979323846;
    for (int i = 0; i < dim; ++i) {
        sum += x[i] * x[i] - 10.0 * cos(2.0 * PI * x[i]);
    }
    return sum;

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)  — parallel safe
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA,
                      mt19937 &rng)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        int I = randInt(rng,1,2);
        newX[j] = x[j] + r * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search) — parallel safe
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t,mt19937 &rng){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population (PARALLEL)
---------------------------------------*/
vector<vector<double>> init_population(vector<mt19937> &rngs){
    vector<vector<double>> pop(POP, vector<double>(DIM));

    #pragma omp parallel
    {
        int tid = omp_get_thread_num();
        mt19937 &local_rng = rngs[tid];

        #pragma omp for schedule(static)
        for(int i=0;i<POP;i++)
            for(int d=0;d<DIM;d++)
                pop[i][d] = randF(local_rng,LB,UB);
    }
    return pop;
}

/* =======================================
             MAIN LOA PARALLEL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    int threads = omp_get_max_threads();
    vector<mt19937> rngs(threads);

    random_device rd;
    for(int i=0;i<threads;i++)
        rngs[i].seed(rd()+i*111);

    vector<vector<double>> pop = init_population(rngs);
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=rastrigin(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
       Full population parallel every step
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        #pragma omp parallel
        {
            int tid = omp_get_thread_num();
            mt19937 &localRng = rngs[tid];

            double localBest = 1e18;
            vector<double> localBestSol(DIM);

            #pragma omp for schedule(static)
            for(int i=0;i<POP;i++){

                vector<int> better;
                for(int j=0;j<POP;j++)
                    if(fitness[j]<fitness[i]) better.push_back(j);

                int betterIdx=-1;
                if(!better.empty())
                    betterIdx = better[randInt(localRng,0,(int)better.size()-1)];

                vector<double> candidate;
                if(randF(localRng,0,1)<0.5 && betterIdx!=-1)
                    candidate = escape(pop[i],pop[betterIdx],localRng);
                else
                    candidate = hide(pop[i],it,localRng);

                double f = rastrigin(candidate);

                if(f<fitness[i]){
                    pop[i]=candidate;
                    fitness[i]=f;
                }
                if(f<localBest){
                    localBest=f;
                    localBestSol=candidate;
                }
            }

            // Update global best safely
            #pragma omp critical
            {
                if(localBest<bestFit){
                    bestFit=localBest;
                    bestSol=localBestSol;
                }
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Rastrigin" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Overwriting rastrigin_omp.cpp


In [ ]:
!g++ -fopenmp rastrigin_omp.cpp -o rastrigin_omp
!./rastrigin_omp

Iter 1000 | Best = 0
Iter 2000 | Best = 0
Iter 3000 | Best = 0
Iter 4000 | Best = 0
Iter 5000 | Best = 0
Iter 6000 | Best = 0
Iter 7000 | Best = 0
Iter 8000 | Best = 0

Final Best Solution:
x1 = 4.28288e-10
x2 = -1.09607e-09
x3 = -1.81896e-09
x4 = 2.1838e-10
x5 = -5.53661e-10

Best Rastrigin Value = 0

Execution Time = 6.03135 sec
